<a href="https://colab.research.google.com/github/pollyaana/-Desktop-App/blob/main/statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import pandas as pd
!python -m spacy download ru_core_news_sm
nlp = spacy.load("ru_core_news_sm")
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = stopwords.words("russian")
!pip install -U pymorphy2==0.9.1
import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='ru')

In [ ]:
posts = {
         'multifunctional': [#'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/karjalankielenkodi.csv',
                      #'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/mediacenter_periodika.csv',
                      #'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/кarelian_flavor.csv',
                     # 'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/karjalanrahvahanliitto.csv',
                      'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/karelian_speakers.csv'],
         }

In [ ]:
def process_text(text, index, df):
    check_hash = False
    processed_parts = []
    for w in nltk.word_tokenize(text):
      if w == '#':
          check_hash = True
          continue
      if check_hash:
          check_hash = False
          continue
      w_tag = morph.parse(w.strip())[0].tag
      if  'Surn' in w_tag or 'Name' in w_tag or 'Patr' in w_tag:
        continue
      if w.isalpha() and w.lower() not in stop_words:
        if w.isupper() and len(w) <= 3:
            processed_parts.append(w)
        else:
            processed_parts.append(morph.parse(w.lower())[0].normal_form)

    result = ' '.join(processed_parts)
    df.at[index, 'tokens'] = str(result)


urls = []
names = []
groups_posts = []
tokens = []
unique_words = []
mid_len_post = []

for key in posts.keys():
    for url in posts[key]:
      urls.append(url)
      names.append(url.split('/')[-1].rstrip('.csv'))
      t = 0
      dict_lem = {}
      df = pd.read_csv('club2562309.csv', usecols = ['text'],encoding='utf-8')
      df = df[df['text'].apply(lambda x: isinstance(x, str))]
      groups_posts.append(len(df))
      for index, row in df.iterrows():
        process_text(row['text'],index, df)
      df.to_excel(url.split('/')[-1].rstrip('.csv')+'_df_tokens.xlsx', index=False)
      df_tokens = df['tokens']
      for row in df_tokens:
        row_split = row.split()

        for lemma in row_split:
          if dict_lem.get(lemma, 0) == 0:
              dict_lem[lemma] = 1
          else:
              dict_lem[lemma] += 1
      data = df['tokens'].tolist()
      for d in data:
        if type(d) == str:
          t += len(d.split())
      mid_len_post.append(t // len(df))
      dict_lem = dict(sorted(dict_lem.items(), key=lambda item: item[1], reverse=True))
      tokens.append(t)
      unique_words.append(len(dict_lem))
      t = 0


In [ ]:
d = {'Группа ВК': names, 'URL': urls, 'Количество постов': groups_posts, 'Количество токенов': tokens, 'Количество уникальных слов': unique_words, 'Средняя длина поста в словах': mid_len_post}
df = pd.DataFrame(data=d)
df.to_csv('statistics.csv', index=False, encoding='utf-8')

In [ ]:
df

,Группа ВК,URL,Количество постов,Количество токенов,Количество уникальных слов,Средняя длина поста в словах
0,karelian_speaker,https://raw.githubusercontent.com/componavt/sn...,3193,173329,40763,54
